In [1]:
import requests
import time
import numpy as np
import pandas as pd
from influxdb import DataFrameClient

In [2]:
now = time.time() # 'UTC'
now

1603393121.8067899

In [3]:
pd.Timestamp.fromtimestamp(now) # 'UTC' Coordinated Universal Time

Timestamp('2020-10-22 18:58:41.806790')

In [4]:
# kl1124 KL571


#if using datetime.datetime.now().tz_localize(tz='Europe/Copenhagen')

pd.Timestamp.fromtimestamp(now).tz_localize(tz='UTC').tz_convert(tz='Europe/Copenhagen') 

Timestamp('2020-10-22 20:58:41.806790+0200', tz='Europe/Copenhagen')

In [5]:
URI = "https://data-live.flightradar24.com/zones/fcgi/feed.js"
params = dict(
    bounds= '56.54,54.61,7.26,17.92',
    faa= 1,
    satellite= 1,
    mlat= 1,
    flarm= 1,
    adsb= 1,
    gnd= 1,
    air= 1,
    vehicles= 1,
    estimated= 1,
    maxage= 14400,
    gliders= 1,
    stats= 1,
)

In [6]:
session = requests.Session()

In [7]:
session.headers.update({'User-Agent':'Pandas'})

In [8]:
columns = {
    0: 'uuid',
    1: 'icao_id',
    2: 'latitude',
    3: 'longitude',
    4: 'track_degree',
    5: 'calibrated_altitude',
    6: 'ground_speed_kts',
    7: 7,
    8: 'receiver_code',
    9: 'aircraft_type',
    10: 'registration',
    11: 'update_time_utc',
    12: 'from',
    13: 'to',
    14: 'flight_number',
    15: 15,
    16: 16,
    17: 'flight_tag',
    18: 18,
    19: 'airline',
}

    

In [9]:
def get_data(params=params):
    
    now = time.time()
    call_time = pd.Timestamp.fromtimestamp(
        now).tz_localize(tz='UTC'
           ).tz_convert(tz='Europe/Copenhagen')
    
    r = session.get(URI, params=params)

    data =r.json()

    _ = [value.insert(0,key) for key, value in data.items() if isinstance(value, list)]

    results = [value for key, value in data.items() if isinstance(value, list)]
    
    df = pd.DataFrame(results).rename(columns, axis=1)
    df = df.replace({'': np.nan})
    df['call_dt'] = call_time
    
    return df
    
    

In [10]:
df = get_data()

In [11]:
df

,uuid,icao_id,latitude,longitude,track_degree,calibrated_altitude,ground_speed_kts,7,receiver_code,aircraft_type,...,update_time_utc,from,to,flight_number,15,16,flight_tag,18,airline,call_dt
0,25d7d049,471F35,55.167,8.802,271,38000,379,5266,F-EKTS1,A21N,...,1603393124,GDN,LPL,W61611,0,0,WZZ244,0,WZZ,2020-10-22 20:58:46.161722+02:00
1,25d7ddbb,4CC0DF,56.302,10.966,116,26000,464,0242,F-ENSN5,B734,...,1603393123,SVG,CPH,BO3163,0,-2816,BCS3163,0,BBD,2020-10-22 20:58:46.161722+02:00
2,25d7e72e,4CAFCD,55.127,11.291,238,35300,363,3512,F-EKAH1,A20N,...,1603393122,CPH,LHR,SK1501,0,1216,SAS1501,0,SAS,2020-10-22 20:58:46.161722+02:00
3,25d7db58,4CAF95,55.402,11.657,55,17775,446,0163,F-EDCG2,CRJ9,...,1603393118,AMS,CPH,SK554,0,-2560,SAS554,0,SAS,2020-10-22 20:58:46.161722+02:00
4,25d7f001,40756C,55.574,12.575,220,3600,179,3517,F-EKMB3,A20N,...,1603393122,CPH,LGW,U28270,0,3264,EZY28HQ,0,EZY,2020-10-22 20:58:46.161722+02:00
5,25d7efbe,4CAF94,55.616,12.640,222,0,35,0275,F-EKCH2,CRJ9,...,1603393107,CPH,GOT,SK446,1,0,SAS446,0,SAS,2020-10-22 20:58:46.161722+02:00
6,25d792bf,48A5F3,55.618,12.647,28,0,27,1203,F-EKCH2,SF34,...,1603393124,AES,CPH,P83273,1,0,SRN3273,0,SRN,2020-10-22 20:58:46.161722+02:00
7,25d7f045,C023C5,55.620,12.663,337,0,8,0403,F-EKCH1,B789,...,1603393123,CPH,NaN,AC7207,1,0,ACA7207,0,ACA,2020-10-22 20:58:46.161722+02:00
8,25d7c494,NaN,55.580,12.889,27,7700,305,2771,F-EKRK1,C680,...,1603393124,NaN,NaN,NaN,0,-3072,C680,0,NaN,2020-10-22 20:58:46.161722+02:00
9,25d7abc7,780BB7,54.900,12.904,62,37000,561,2330,F-EKRK1,B77L,...,1603393121,MAD,HGH,CAO1028,0,0,CAO1028,0,CAO,2020-10-22 20:58:46.161722+02:00


In [ ]:
<div class="content-area F-EKMB3">
        <p class="adsb">
            Terrestrial based tracking with very high levels of GPS accuracy for most aircraft. Some older
            aircraft use inertial reference units instead of GPS for positions, decreasing positional accuracy.
            <a target="_blank" href="/how-it-works"><span>Read more<svg><use xlink:href="#icon-arrow"></use></svg></span></a>
        </p>
        <p class="fsat tsat">
            Satellite based tracking with very high levels of GPS accuracy for most aircraft. Some older aircraft use
            inertial reference units instead of GPS for positions, decreasing positional accuracy.
            <a target="_blank" href="/how-it-works"><span>Read more<svg><use xlink:href="#icon-arrow"></use></svg></span></a>
        </p>
        <p class="mlat">
            Terrestrial based tracking using the time difference of arrival to calculate position. Should
            give high positional accuracy during most phases of flight, but position errors can sometimes occur. The
            ground speed is calculated and can sometimes be incorrect, especially during turns and at low altitudes.
            Vertical speed is also calculated, so errors can sometimes occur. Altitude data come from the transponder
            and should be correct.
            <a target="_blank" href="/how-it-works"><span>Read more<svg><use xlink:href="#icon-arrow"></use></svg></span></a>
        </p>
        <p class="flarm">
            Terrestrial based tracking used in light aircraft, such as gliders. Variable levels of data
            quality based on limited tracking capabilities for gliders.
            <a target="_blank" href="/how-it-works"><span>Read more<svg><use xlink:href="#icon-arrow"></use></svg></span></a>
        </p>
        <p class="faa">
            3rd party data covering North America and certain oceanic regions around North America,
            aggregated from different data sources. Provided to Flightradar24 as is with variable quality based on
            multiple factors.
            <a target="_blank" href="/how-it-works"><span>Read more<svg><use xlink:href="#icon-arrow"></use></svg></span></a>
        </p>
        <p class="est">
            An aircraft's position may be estimated for up to 240 minutes after coverage has been lost based
            on the great circle path between the last received position and destination. Accuracy of estimated positions
            will vary based on time since last received position and flight route.
            <a target="_blank" href="/how-it-works"><span>Read more<svg><use xlink:href="#icon-arrow"></use></svg></span></a>
        </p>
        <p class="inv">
            This aircraft is broadcasting an invalid transponder code due to malfunction or incorrect
            programming. As the ICAO 24-bit address is incorrect, it's often impossible to identify the aircraft.
            <a target="_blank" href="/how-it-works"><span>Read more<svg><use xlink:href="#icon-arrow"></use></svg></span></a>
        </p>
        <ul>
            <li>
                <svg>
                    <use xlink:href="#icon-radar"></use>
                </svg>
                <div>
                    <span>F-EKMB3</span>
                    <p>Receiver code displayed is randomly chosen from all receivers currently seeing the aircraft.</p>
                </div>
            </li>
        </ul>
        <a target="_blank" class="coverage" href="/apply-for-receiver">
            <span>
                Increase the coverage in your area <svg class="arrow"><use xlink:href="#icon-fr24-arrow-3px"></use></svg>
            </span>
        </a>
    </div>

In [12]:
user = 'danpra'
password = 'influxdbpwd'
host = 'influxdb'
port = 8086
dbname= 'airlines'
protocol = 'line'

client = DataFrameClient(host, port, user, password, dbname)

In [18]:
df.tail()

,uuid,icao_id,latitude,longitude,track_degree,calibrated_altitude,ground_speed_kts,7,receiver_code,aircraft_type,registration,update_time_utc,from,to,flight_number,15,16,flight_tag,18,airline
call_dt,,,,,,,,,,,,,,,,,,,,
2020-10-22 20:58:46.161722+02:00,25d7cf6c,4CA9ED,55.173,14.800,274,40000,369,1165,F-EKRN2,B738,EI-EVP,1603393121,VNO,LPL,FR9621,0,0,RYR6WG,0,RYR
2020-10-22 20:58:46.161722+02:00,25d7cdfb,440102,55.229,16.946,206,34050,360,4533,F-EPSY2,B734,OE-IBL,1603393123,TLL,MUC,3V4537,0,0,TAY4537,0,TAY
2020-10-22 20:58:46.161722+02:00,25d7b49d,NaN,55.341,17.112,259,40000,375,3526,F-ESMS4,LJ60,NaN,1603393123,NaN,NaN,NaN,0,-64,LJ60,0,NaN
2020-10-22 20:58:46.161722+02:00,25d7451c,3C7069,54.635,17.697,214,32025,398,4523,F-EDCG2,MD11,D-ALCI,1603393123,OVB,FRA,LH8391,0,0,GEC8391,0,GEC
2020-10-22 20:58:46.161722+02:00,25d7c8b5,341119,55.361,17.764,148,22750,397,4527,F-ESSV5,B752,EC-FTR,1603393123,TLL,GDN,RGN3789,0,-2240,BCS3789,0,RGN


In [15]:
df.set_index('call_dt', drop=True, inplace=True)

In [20]:
df.columns

Index([               'uuid',             'icao_id',            'latitude',
                 'longitude',        'track_degree', 'calibrated_altitude',
          'ground_speed_kts',                     7,       'receiver_code',
             'aircraft_type',        'registration',     'update_time_utc',
                      'from',                  'to',       'flight_number',
                          15,                    16,          'flight_tag',
                          18,             'airline'],
      dtype='object')

In [41]:
df.dropna(inplace=True)

In [42]:
time_value = df[['calibrated_altitude']]

In [43]:
time_value.index

DatetimeIndex(['2020-10-22 20:58:46.161722+02:00',
               '2020-10-22 20:58:46.161722+02:00',
               '2020-10-22 20:58:46.161722+02:00',
               '2020-10-22 20:58:46.161722+02:00',
               '2020-10-22 20:58:46.161722+02:00',
               '2020-10-22 20:58:46.161722+02:00',
               '2020-10-22 20:58:46.161722+02:00',
               '2020-10-22 20:58:46.161722+02:00',
               '2020-10-22 20:58:46.161722+02:00',
               '2020-10-22 20:58:46.161722+02:00',
               '2020-10-22 20:58:46.161722+02:00',
               '2020-10-22 20:58:46.161722+02:00',
               '2020-10-22 20:58:46.161722+02:00',
               '2020-10-22 20:58:46.161722+02:00'],
              dtype='datetime64[ns, Europe/Copenhagen]', name='call_dt', freq=None)

In [49]:
tags = ['uuid',
'icao_id',
'latitude',
'longitude',
'track_degree',
'ground_speed_kts',
'from',
'to',
'flight_number',
'flight_tag',
'airline']

In [57]:
tags_ = {tag: df[[tag]] for tag in tags }
df = df[tags]

In [46]:
print(f"Create database: {dbname}")
client.create_database(dbname)

Create database: airlines


In [62]:
print("Write DataFrame with Tags")
client.write_points(df, 'my_data')

Write DataFrame with Tags


True

In [63]:
print("Read DataFrame")
client.query(f"select * from my_data")

Read DataFrame


defaultdict(list,
            {'my_data':                                  airline flight_number flight_tag from  \
             2020-10-22 18:58:46.161722+00:00     RGN       RGN3789    BCS3789  TLL   
             
                                               ground_speed_kts icao_id  latitude  \
             2020-10-22 18:58:46.161722+00:00               397  341119    55.361   
             
                                               longitude   to  track_degree      uuid  
             2020-10-22 18:58:46.161722+00:00     17.764  GDN           148  25d7c8b5  })